<a href="https://colab.research.google.com/github/Vakhranev/BraPoRus/blob/main/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%81%D1%83%D0%B1%D1%82%D0%B8%D1%82%D1%80%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def preprocess_file(file_path, output_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    processed_lines = []
    for i, line in enumerate(lines):
        # Проверяем, начинается ли строка с "EXP" или "EAK-V"
        if line.startswith("EXP") or line.startswith("EAK-V"):
            # Проверяем, есть ли перед ней пустая строка
            if i > 0 and lines[i - 1].strip() != "":
                processed_lines.append("")  # Добавляем пустую строку
        processed_lines.append(line.strip())  # Добавляем текущую строку

    # Сохраняем обработанный текст
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write("\n".join(processed_lines))


# Пример вызова функции
input_file = "EAK-V_59_f_2023_06_17_S1_audio.m4a_REV_GEORGYB.txt"
output_file = "EAK-V_59_f_2023_06_17_S1_audio.m4a_REV_GEORGYB_preprocessed.txt"

preprocess_file(input_file, output_file)

In [12]:
import re

# Функция для парсинга реплик из файла 1
def parse_replicas_file1(file):
    replicas = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            if ']' in line:
                parts = line.split('] ')
                bracket_info = parts[0].strip() + "]"  # Информация в квадратных скобках
                speaker_info = bracket_info.split()[0]  # Спикер
                bracket_info = bracket_info.split()[1]
                text = parts[1].strip()  # Текст реплики
                replicas.append((speaker_info, bracket_info, text))
    return replicas

# Функция для парсинга реплик из файла 2
def parse_replicas_file2(file):
    replicas = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            # Ищем реплики в файле 2 (после "EXP" или "EAK-V", затем двоеточие и пробел)
            match = re.match(r"^(EXP|EAK-V)[^:]*: (.+)", line.strip())
            if match:
                speaker = match.group(1)  # "EXP" или "EAK-V"
                text = match.group(2)  # Текст реплики
                replicas.append((speaker, text))
    return replicas

# Функция для сравнения токенов (первые или последние три)
def compare_tokens(tokens1, tokens2):
    return tokens1 == tokens2

# Функция для поиска токенов text2 в списке text1
def tokens_in_texts(tokens, texts):
    tokens_set = set(tokens)
    for _, text in texts:
        text_tokens = set(text.lower().split())
        if tokens_set.issubset(text_tokens):
            return True
    return False

# Функция для поиска следующего тайм-кода по условиям
def find_next_timecode(replicas_file1, last_timecode, speaker_condition):
    found_last = False
    for speaker_info, bracket_info, text in replicas_file1:
        # Ищем следующий тайм-код после last_timecode
        if found_last:
            # Проверяем условие для спикера
            if speaker_condition in speaker_info:
                return bracket_info
        if bracket_info == last_timecode:
            found_last = True
    return None  # Если тайм-код не найден

# Функция для обработки файлов и вывода совпадений
def process_files(file1, file2):
    collection = []  # Список для хранения результата
    last_published_timecode = None  # Последний опубликованный тайм-код
    replicas_file1 = parse_replicas_file1(file1)
    replicas_file2 = parse_replicas_file2(file2)

    for speaker, text2 in replicas_file2:
        matched = False

        # Поиск совпадений по первым трём токенам
        tokens2_start = text2.lower().split()[:3]
        for speaker_info, bracket_info, text1 in replicas_file1:
            tokens1_start = text1.lower().split()[:3]
            if compare_tokens(tokens1_start, tokens2_start):
                # Совпадение по первым трём токенам
                collection.append(f"{speaker}: {bracket_info} {text2}")
                last_published_timecode = bracket_info  # Запоминаем последний опубликованный тайм-код
                matched = True
                break

        if not matched:
            # Поиск совпадений по последним трём токенам
            tokens2_end = text2.lower().split()[-3:]
            for speaker_info, bracket_info, text1 in replicas_file1:
                tokens1_end = text1.lower().split()[-3:]
                if compare_tokens(tokens1_end, tokens2_end):
                    # Совпадение по последним трём токенам
                    # Определяем условие для спикера
                    if speaker == "EXP":
                        speaker_condition = "Speaker1" or "Speaker3"
                    elif speaker == "EAK-V":
                        speaker_condition = "Speaker2" or "Speaker3"

                    # Найти следующий тайм-код с учетом условия
                    next_timecode = find_next_timecode(replicas_file1, last_published_timecode, speaker_condition)
                    if next_timecode:
                        collection.append(f"{speaker}: {next_timecode} {text2}")
                        last_published_timecode = next_timecode  # Обновляем последний опубликованный тайм-код
                        matched = True
                        break

        if not matched:
            # Если ни одно из совпадений не найдено, добавляем реплику как есть
            collection.append(f"{speaker}: {text2}")

    # Выводим результат в порядке файла 2
    for line in collection:
        print(line)
    output_file = 'output.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        for line in collection:
            f.write(line + '\n')
    return 'output.txt'

# Пример использования
process_files('EAK-V_59_f_2023_06_17_S1_audio.txt',
              'EAK-V_59_f_2023_06_17_S1_audio.m4a_REV_GEORGYB_preprocessed.txt')

EXP: [00:00:07] Я хотела попросить вас рассказать историю вашей семьи.
EAK-V: Окей.
EXP: Расскажите все, что вы считаете наиболее важным, о ваших бабушках и дедушках, родителях, кем они были и откуда родом? Что вы знаете об их жизни в Бразилии и после переезда сюда? Как вы жили здесь? Как сложилась ваша жизнь? Я хотела бы услышать все, что вы посчитаете самым важным. И постараюсь не мешать вашему рассказу.
EAK-V: [00:00:44] Прекрасно. Так. Со стороны отца: они из Астрахани так же, как… и дед Александр. Его супруга — это Матрона. Они Калинины. Как это… рыбаки… выехали… имели три лодки рыбачьей в Астрахани, выехали через Иран, потому что коммунистическая власть их преследовала. Они оставили одного сына, потому что он был болен. Так, после этого они в Иране прожили до того, что началось время, ????? [1’27”], их преследовали. Мой отец вернулся в Россию, разыскивать брата. Оттуда он пробыл некоторое время, не могу сказать сколько времени там он пробыл. Сумел выехать в Бразилию, накопить ден

'output.txt'